In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data = pd.read_csv("/home/aditya/Downloads/Groceries_dataset.csv")
data

In [ ]:
data.info()

In [ ]:
# Check availability of null values

data.isnull().sum().sort_values( ascending=False )

In [ ]:
data['date'] = pd.to_datetime( data['Date'] )
data.info()

In [ ]:
data

In [ ]:
grouped = data.groupby( by = 'itemDescription' ).size().reset_index(name='frequency').sort_values( by = 'frequency', ascending=False )
grouped

In [ ]:
bars = grouped['itemDescription']
height = grouped['frequency']


In [ ]:
x_position = np.arange(len(bars))
x_position

In [ ]:
plt.figure( figsize=(160,80) )
plt.bar(x_position, height, color=(0.2, 0.3, 0.5, 0.5) )
plt.title("Top 10 Sold Items")
plt.xlabel("Item Names")
plt.ylabel("No. of Quantity Sold")
plt.xticks(x_position, bars)
plt.show()

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
date_df = data.set_index('Date')


In [ ]:
date_df.resample("M")["itemDescription"].count().plot(
    figsize=(20,8),
    grid=True,
    title='No. of items sold by month'
).set(xlabel='Date', ylabel='No. of items sold');


In [ ]:
cust_level = data[['Member_number', 'itemDescription']].sort_values( by='Member_number', ascending=False )
cust_level['itemDescription'] = cust_level['itemDescription'].map(str)
cust_level

In [ ]:
transactions = [ a[1]['itemDescription'].tolist() 
                for a in list(cust_level.groupby([ "Member_number" ]) ) 
               ]

transactions

In [ ]:
!pip install apyori


In [ ]:
from apyori import apriori
rules = apriori( transactions = transactions, min_support=0.002 , min_confidence=0.05, min_lift=3, min_length=2 ) 

In [ ]:
results = list(rules)
results

In [ ]:
def inspect(results):
    lhs = [ tuple(result[2][0][0])[0] for result in results ]

    rhs = [ tuple(result[2][0][1])[0] for result in results ]

    support = [ result[1] for result in results ]

    confidences = [ result[2][0][2] for result in results ]

    lifts = [ result[2][0][3] for result in results ]

    return list(zip(lhs, rhs, support, confidences, lifts))



In [ ]:
results_df = pd.DataFrame( inspect(results), columns=['Customer Buyed', 'Recommendation', 'Support', 'Confidences', 'Lifts'] )
results_df.nlargest(n=10, columns='Lifts')